# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_info = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_info.head(20)


,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed
0,0,Henties Bay,43,NaN,1593394586,76,-22.12,14.28,55.83,3.98
1,1,Bethel,20,US,1593394586,94,41.37,-73.41,70.00,0.72
2,2,Saint-François,90,FR,1593394586,82,46.42,3.91,60.80,2.24
3,3,Ushuaia,40,AR,1593394436,69,-54.80,-68.30,35.60,10.29
4,4,Mar del Plata,96,AR,1593394343,93,-38.00,-57.56,48.00,5.53
5,5,Kruisfontein,0,ZA,1593394587,80,-34.00,24.73,49.28,10.74
6,6,Brønnøysund,63,NO,1593394588,52,65.46,12.20,68.00,12.75
7,7,Yumen,0,CN,1593394588,22,40.28,97.20,80.67,13.60
8,8,Albany,0,US,1593394503,91,42.60,-73.97,69.01,7.56
9,9,Dikson,99,RU,1593394588,99,73.51,80.55,42.84,22.03


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
gmaps.configure(api_key=g_key)

locations = city_info[["Latitude", "Longitude"]]
weight = city_info["Humidity"]
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
hotel_df = city_info.loc[(city_info["Max_Temp"]>70)
                     &(city_info["Max_Temp"]<80)
                     &(city_info["Wind_Speed"]<10)
                     &(city_info["Cloudiness"]==0)
                     &(city_info["Humidity"]<80),:]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed
36,36,Marrakesh,0,MA,1593394416,60,31.63,-8.01,75.20,5.82
184,184,Follonica,0,IT,1593394624,62,42.93,10.76,72.00,3.20
197,197,Mount Isa,0,AU,1593394625,33,-20.73,139.50,73.40,6.93
255,255,Tongchuanshi,0,CN,1593394637,58,35.08,109.09,78.12,5.84
279,279,Nola,0,IT,1593394643,75,40.92,14.53,71.01,0.74
355,355,Corinth,0,GR,1593394657,46,37.94,22.96,73.99,1.99
434,434,Sabha,0,LY,1593394677,22,27.04,14.43,79.93,9.40
441,441,Gilgit,0,PK,1593394678,28,35.92,74.31,71.74,2.55
498,498,São Miguel do Araguaia,0,BR,1593394687,49,-13.28,-50.16,72.28,5.19
518,518,Waddān,0,LY,1593394691,35,29.16,16.14,73.99,8.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df["Hotel_Name"]=""


C:\Users\Amber\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_Temp,Wind_Speed,Hotel_Name
36,36,Marrakesh,0,MA,1593394416,60,31.63,-8.01,75.20,5.82,
184,184,Follonica,0,IT,1593394624,62,42.93,10.76,72.00,3.20,
197,197,Mount Isa,0,AU,1593394625,33,-20.73,139.50,73.40,6.93,
255,255,Tongchuanshi,0,CN,1593394637,58,35.08,109.09,78.12,5.84,
279,279,Nola,0,IT,1593394643,75,40.92,14.53,71.01,0.74,


In [46]:

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
marker_locations=[]

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params["location"] = f"{lat},{lng}"
    # run a request using our params dictionary
    hotel_search = requests.get(base_url, params=params)
    hotel_search=hotel_search.json()
    hotel_df.loc[index, "Hotel_Name"] = hotel_search["results"][0]["name"]
     # Create a map using state centroid coordinates to set markers
    marker_lat = hotel_search["results"][0]["geometry"]["location"]["lat"]
    marker_lng = hotel_search["results"][0]["geometry"]["location"]["lng"]
    marker_locations=(f"{marker_lat},{marker_lng}")
 
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

# fig = gmaps.figure()

# fig.add_layer(heat_layer)
# fig.add_layer(bank_layer)



AttributeError: 'str' object has no attribute 'astype'

In [47]:
target_radius = 5000
loc = "31.63,-8.01"
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key,
    "location":loc
}
response = requests.get(base_url, params=params)
response=response.json()

marker_lat = response["results"][0]["geometry"]["location"]["lat"]
marker_lng = response["results"][0]["geometry"]["location"]["lng"]
marker_locations = loc
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
